## Elements of functional programming

Немного об этом от самого [Гвидо](http://python-history.blogspot.com/2009/04/origins-of-pythons-functional-features.html)

In [1]:
from operator import itemgetter, attrgetter, methodcaller

**itemgetter**

In [2]:
some_data = [
    ('A', 'JP', 36.933, (1, 139.691667)),
    ('B', 'IN', 21.935, (28.613889, 77.208889)),
    ('C', 'MX', 20.142, (19.433333, -99.133333)),
    ('A', 'US', 20.104, (-1, -74.020386)),
    ('B', 'BR', 19.649, (-23.547778, -46.635833)),
]

In [8]:
for city in sorted(some_data, key=itemgetter(0)):
    print(city)

('A', 'JP', 36.933, (1, 139.691667))
('A', 'US', 20.104, (-1, -74.020386))
('B', 'IN', 21.935, (28.613889, 77.208889))
('B', 'BR', 19.649, (-23.547778, -46.635833))
('C', 'MX', 20.142, (19.433333, -99.133333))


In [10]:
for city in sorted(some_data, key=itemgetter(0, 2)):
    print(city)

('A', 'US', 20.104, (-1, -74.020386))
('A', 'JP', 36.933, (1, 139.691667))
('B', 'BR', 19.649, (-23.547778, -46.635833))
('B', 'IN', 21.935, (28.613889, 77.208889))
('C', 'MX', 20.142, (19.433333, -99.133333))


Получили сортировку по нескольким параметрам. Также можно написать и с помощью лямбды

In [11]:
name = itemgetter(0, 1)  # uses __getitem__

for city in some_data:
    print(name(city))

('A', 'JP')
('B', 'IN')
('C', 'MX')
('A', 'US')
('B', 'BR')


**attrgetter**

In [12]:
from collections import namedtuple

In [13]:
Subject = namedtuple('Subject', 'name difficulty')

In [14]:
python_course = Subject('Python', 1)
python_course

Subject(name='Python', difficulty=1)

In [19]:
python_course.name

'Python'

In [20]:
sub_name = attrgetter('name')

In [22]:
sub_name(python_course)

'Python'

**methodcaller**

In [26]:
sample_str = 'This is really functional'

In [27]:
upcase = methodcaller('upper')
upcase(sample_str)

'THIS IS REALLY FUNCTIONAL'

In [29]:
repl = methodcaller('replace', ' ', '!')
repl(sample_str)

'This!is!really!functional'

И еще много подобных методов

**Functools.partial**

Иногда бывает, что в конкретной ситуации вы хотите вызывать функцию с фиксированным конкретным параметром, у которого нет дефолтного значения

In [30]:
from operator import mul
from functools import partial

In [31]:
mul(4, 3)

12

In [32]:
quadriple = partial(mul, 4)

In [35]:
quadriple(10)

40

In [36]:
list(map(quadriple, range(10)))

[0, 4, 8, 12, 16, 20, 24, 28, 32, 36]

Мы не могли бы использовать `mul` в `map` без такой модификации

Callable objects with class example

## Анонимные функции

Ключевое слово **lambda**, краткое объявление функции как выражения.

Отлично может использоваться в качестве аргумента для функций высших порядков

In [41]:
sample_list = [1, '31412412412', (1, 2,), (0,)]

In [42]:
sorted(sample_list, key=lambda element: len(str(element)))

[1, (0,), (1, 2), '31412412412']

Вспомним пример с listcomp

In [43]:
symbols = '$¢£¥€¤'
filtered_symbols = [s for s in symbols if ord(s) > 200]
filtered_symbols

['€']

Можно использовать функциональный стиль и написать это же выражение через `filter`

In [46]:
for elem in filter(lambda elem: ord(elem) > 200, symbols):
    print(elem)

€


In [47]:
tuple(filter(lambda elem: ord(elem) > 200, symbols))

('€',)

In [52]:
a = ['1234124124', '42', '2', '', '1', 'asdf']

list(filter(None, a))

['1234124124', '42', '2', '1', 'asdf']

## Generators

Мы уже упоминали generator expressions, а теперь можем сделать такой и сами

In [53]:
def my_custom_range(start, end):
    while start < end:
        yield start
        start += 1

In [54]:
my_custom_range(0, 10)

<generator object my_custom_range at 0x7f4ae3389eb0>

In [61]:
for i in my_custom_range(0, 10):
    print(i)

0
1
2
3
4
5
6
7
8
9


In [62]:
list(my_custom_range(0, 10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

## Decorators

Декоратор -- обертка над функцией. Позволяет сделать что-то до и после вызова передаваемой функции.

Для этого функции декоратора передается декорируемая функция, а внутри создается новая, которая будет выполняться вместе с декорируемой. Ей передаются такие же аргументы, и в ней же пишется код того, что будет совершаться до и после запуска переданной функции, а между этими блоками будет запускаться сама декорируемая функция. Внешняя функция возвращает ссылку на получившуюся функцию

### Простой пример

In [63]:
def my_decorator(func):
    def decorated(a, b):  # название можно менять
        print("doing smth before")
        func(a, b)
        print("doing smth after")
    return decorated

def func(a, b):
    print(a + b)

func = my_decorator(func)
func(2, 3)

doing smth before
5
doing smth after


In [64]:
@my_decorator
def new_func(a, b):
    print(a + b)

In [65]:
new_func(1, 2)

doing smth before
3
doing smth after


**Задачка**

Попробуйте реализовать декоратор, который будет приводить выход функции к верхнему регистру

In [72]:
def upper_decorator(func):
    def decorated():
        value = func()
        return value.upper()
    
    return decorated

In [73]:
@upper_decorator
def print_something():
    return 'today is a good day'

In [74]:
print_something()

'TODAY IS A GOOD DAY'

### О сохранении поведения help и докстроки

In [87]:
def do_first_function():
    """Docstring for the first function"""
    
    print("I am always first!")

In [88]:
do_first_function.__name__, do_first_function.__doc__

('do_first_function', 'Docstring for the first function')

In [89]:
help(do_first_function)

Help on function do_first_function in module __main__:

do_first_function()
    Docstring for the first function



In [91]:
def decorator(func):
    def wrapper(*args, **kwargs):
        """Some wrapper function"""
        func()
    return wrapper

@decorator
def do_first_function():
    """Docstring for the first function"""
    print("I am always first!")

@decorator
def do_second_function():
    """Docstring for the second function"""
    print("I am always second!")

print(do_first_function.__name__)
print(do_first_function.__doc__)
print(do_second_function.__name__)
print(do_second_function.__doc__)

wrapper
Some wrapper function
wrapper
Some wrapper function


In [93]:
help(do_first_function)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)
    Some wrapper function



In [95]:
help(do_second_function)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)
    Some wrapper function



Кажется, мы потеряли "документацию". Можно попробовать поправить вручную!

In [96]:
def decorator(func):
    def wrapper(*args, **kwargs):
        """Some wrapper function"""
        func()
        
    wrapper.__name__ = func.__name__
    wrapper.__doc__ = func.__doc__
    return wrapper

@decorator
def do_first_function():
    """Docstring for the first function"""
    print("I am always first!")

@decorator
def do_second_function():
    """Docstring for the second function"""
    print("I am always second!")

print(do_first_function.__name__)
print(do_first_function.__doc__)
print(do_second_function.__name__)
print(do_second_function.__doc__)

do_first_function
Docstring for the first function
do_second_function
Docstring for the second function


In [97]:
help(do_first_function)

Help on function do_first_function in module __main__:

do_first_function(*args, **kwargs)
    Docstring for the first function



In [99]:
help(do_second_function)

Help on function do_second_function in module __main__:

do_second_function(*args, **kwargs)
    Docstring for the second function



**все ли в порядке?**

In [100]:
from functools import wraps

def decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        """Some wrapper function"""
        func()
    return wrapper

@decorator
def do_first_function():
    """Docstring for the first function"""
    print("I am always first!")

@decorator
def do_second_function():
    """Docstring for the second function"""
    print("I am always second!")

In [101]:
print(do_first_function.__name__)
print(do_first_function.__doc__)
print(do_second_function.__name__)
print(do_second_function.__doc__)

do_first_function
Docstring for the first function
do_second_function
Docstring for the second function


In [102]:
help(do_first_function)

Help on function do_first_function in module __main__:

do_first_function()
    Docstring for the first function



In [103]:
help(do_second_function)

Help on function do_second_function in module __main__:

do_second_function()
    Docstring for the second function



### Пример. Декоратор-время

Подумайте, как можно реализовать декоратор, который будет выводить время работы функции?

Для замера времени вам поможет функция `time` из модуля `time`

In [106]:
import time

In [105]:
def calc_slow_fact(n):
    time.sleep(1)
    
    if n < 0:
        return -1
    
    if n <= 1:
        return 1
    
    return n * calc_slow_fact(n - 1)

In [107]:
calc_slow_fact(4)

24

In [113]:
def clock(func):
    @wraps(func)
    def clocked(*args):
        t_start = time.time()
        res = func(*args)
        total_time = time.time() - t_start
        print(f'{func.__name__}({args}) -> {res} executed in {total_time:.2f}s')
        return res
    return clocked

In [114]:
@clock
def calc_slow_fact(n):
    time.sleep(1)
    if n < 0:
        return -1
    
    if n <= 1:
        return 1
    return n * calc_slow_fact(n - 1)

In [116]:
calc_slow_fact(10)

calc_slow_fact((1,)) -> 1 executed in 1.00s
calc_slow_fact((2,)) -> 2 executed in 2.00s
calc_slow_fact((3,)) -> 6 executed in 3.00s
calc_slow_fact((4,)) -> 24 executed in 4.00s
calc_slow_fact((5,)) -> 120 executed in 5.01s
calc_slow_fact((6,)) -> 720 executed in 6.01s
calc_slow_fact((7,)) -> 5040 executed in 7.01s
calc_slow_fact((8,)) -> 40320 executed in 8.01s
calc_slow_fact((9,)) -> 362880 executed in 9.01s
calc_slow_fact((10,)) -> 3628800 executed in 10.01s


3628800

### Параметр для декоратора

Модифицируем декоратор из примера выше, чтобы сделать его отключаемым

In [24]:
def clock(active=False):
    def decorate(func):
        
        @wraps(func)
        def clocked(*args):
            if not active:
                return func(*args)
            
            t_start = time.time()
            res = func(*args)
            total_time = time.time() - t_start
            print(f'{func.__name__}({args}) -> {res} executed in {total_time:.2f}s')
            return res
        
        return clocked
    return decorate

In [118]:
@clock(active=False)
def calc_slow_fact(n):
    time.sleep(1)
    if n < 0:
        return -1
    
    if n <= 1:
        return 1
    return n * calc_slow_fact(n - 1)

In [119]:
calc_slow_fact(2)

2

In [120]:
@clock(active=True)
def calc_slow_fact(n):
    time.sleep(1)
    if n < 0:
        return -1
    
    if n <= 1:
        return 1
    return n * calc_slow_fact(n - 1)

In [121]:
calc_slow_fact(2)

calc_slow_fact((1,)) -> 1 executed in 1.00s
calc_slow_fact((2,)) -> 2 executed in 2.00s


2

### LRU Cache

In [122]:
from functools import lru_cache

In [132]:
@clock(active=True)
def calc_fib(n):
    time.sleep(1)
    
    if n < 2:
        return n
    
    return calc_fib(n - 2) + calc_fib(n - 1)

In [133]:
calc_fib(4)

calc_fib((0,)) -> 0 executed in 1.00s
calc_fib((1,)) -> 1 executed in 1.00s
calc_fib((2,)) -> 1 executed in 3.00s
calc_fib((1,)) -> 1 executed in 1.00s
calc_fib((0,)) -> 0 executed in 1.00s
calc_fib((1,)) -> 1 executed in 1.00s
calc_fib((2,)) -> 1 executed in 3.00s
calc_fib((3,)) -> 2 executed in 5.01s
calc_fib((4,)) -> 3 executed in 9.01s


3

In [136]:
@clock(active=True)
@lru_cache(None)
def calc_fib(n):
    time.sleep(1)
    
    if n < 2:
        return n
    
    return calc_fib(n - 2) + calc_fib(n - 1)

In [137]:
calc_fib(4)

calc_fib((0,)) -> 0 executed in 1.00s
calc_fib((1,)) -> 1 executed in 1.00s
calc_fib((2,)) -> 1 executed in 3.00s
calc_fib((1,)) -> 1 executed in 0.00s
calc_fib((2,)) -> 1 executed in 0.00s
calc_fib((3,)) -> 2 executed in 1.00s
calc_fib((4,)) -> 3 executed in 5.01s


3